In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
 
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
# hyper parameter
LEARNING_RATE = 0.005
N_EPOCHS = 100
N_BATCHS = 100
N_CLASS = 10
DROPOUT_RATE = 0.3

In [ ]:
label = ['비행기','자동차','새','고양이','사슴','개','개구리','말','배','트럭']
label_eng = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# 다운로드 받는다.
(train_image, train_label), (test_image, test_label) = keras.datasets.cifar10.load_data()

In [ ]:
# 32 * 32 * 3
train_image.shape, test_image.shape

In [ ]:
N_TRAIN = train_image.shape[0]
N_TEST = test_image.shape[0]

In [ ]:
np.unique(train_label, return_counts=True)

In [ ]:
train_label[:5]

In [ ]:
import matplotlib.pyplot as plt
for i in range(0,15):
    plt.subplot(3,5, i+1)
    plt.imshow(X_train[i])
    plt.axis('off')
    plt.title(f'{y_train[i]}-{label_eng[y_train[i,0]]}')
plt.show()    

# Dataset 생성

In [ ]:
# pixel값을 0 ~ 1 사이로 조정
X_train = train_image.astype(np.float32)/255.
X_test = test_image.astype(np.float32) /255.

# y - onehot encoding
y_train = keras.utils.to_categorical(train_label, num_classes=10)
y_test = keras.utils.to_categorical(test_label, 10) 

In [ ]:
# dataset 구성
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))\
                               .shuffle(buffer_size=N_TRAIN)\
                               .batch(N_BATCHS,drop_remainder=True).repeat()
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))\
                              .batch(N_BATCHS)                        

## 모델 구성

In [ ]:
from tensorflow.keras import layers
def create_model():
    model = keras.Sequential()
    #Conv Layer
    model.add(layers.Input((32,32,3))) #InputLayer(input_shape=(32,32,3)) : 동일
    
    model.add(layers.Conv2D(filters=32, kernel_size=3, padding='SAME'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    
    model.add(layers.Conv2D(filters=32, kernel_size=3, padding='SAME'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))

    model.add(layers.MaxPooling2D(padding='SAME'))

    model.add(layers.Conv2D(filters=64, kernel_size=3, padding='SAME'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    
    model.add(layers.Conv2D(filters=64, kernel_size=3, padding='SAME'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))

    model.add(layers.MaxPooling2D(padding='SAME'))
    
    model.add(layers.Conv2D(filters=128, kernel_size=3, padding='SAME'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    
    model.add(layers.Conv2D(filters=128, kernel_size=3, padding='SAME'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))

    model.add(layers.MaxPooling2D(padding='SAME'))

 
    # Full Connected Layer
    model.add(layers.Flatten())
    model.add(layers.Dropout(DROPOUT_RATE))
    model.add(layers.Dense(512))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())    
    model.add(layers.Dense(10, activation='softmax'))
    return model

In [ ]:
# model = create_model()
model = create_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

In [ ]:
keras.utils.plot_model(model, to_file='model_shapes.png', show_shapes=True)

In [ ]:
# 학습 전에 테스트
model.evaluate(test_dataset)

In [ ]:
# 학습시 전달할 매개변수
steps_per_epoch = N_TRAIN//N_BATCHS
validation_steps = int(np.ceil(N_TEST/N_BATCHS))
print(steps_per_epoch, validation_steps)

In [ ]:
import os
base_dir = '/content/drive/My Drive/save_models'
save_path = os.path.join(base_dir, 'cifar10_best_model')
save_path

In [ ]:
import time
start = time.time()
modelcheckpoint_callback = keras.callbacks.ModelCheckpoint(filepath=save_path, monitor='val_loss', save_best_only=True)

history = model.fit(train_dataset, 
                    epochs=N_BATCHS,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=test_dataset,
                    validation_steps=validation_steps)
                    callbacks=[modelcheckpoint_callback])
end = time.time()

In [ ]:
(end-start)/60

In [ ]:
saved_model = keras.models.load_model(save_path)

In [ ]:
saved_model.evaluate(test_dataset)

In [ ]:
history.history.keys()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.plot(range(N_EPOCHS+1), history.history['loss'], label='train loss')
plt.plot(range(N_EPOCHS+1), history.history['val_loss'], label='val loss')
plt.legend()
plt.title('loss')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.plot(range(N_EPOCHS+1), history.history['accuracy'], label='train accuracy')
plt.plot(range(N_EPOCHS+1), history.history['val_accuracy'], label='val accuracy')
plt.legend()
plt.show()